In [2]:
# Libraries used

from itertools import combinations
from collections import defaultdict
import time
import pandas as pd

In [5]:
# Retail data set
dataset = pd.read_csv('retail.dat.txt', sep='delimiter', header=None, engine='python')
dataset.columns = ['items']
df_items = dataset['items']
df = df_items.apply(lambda x: x.split(' '))

In [4]:
# Netflix data set
dataset = pd.read_csv('netflix.data', sep='delimiter', header=None, engine='python')
dataset.columns = ['items']
df_items = dataset['items']
df = df_items.apply(lambda x: x.split(' '))

In [7]:
# Apriori random sampling

def apriorirandom(df, min_support=0.01, perc=0.2):
    itemsets = {}
    # random sampling
    df_sampled = df.sample(frac=perc)
    transactions = list(map(set, df_sampled))
    num_transactions = len(transactions)
    min_support_count = min_support * num_transactions

    # Making C1
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            if item not in item_counts:
                item_counts[item] = 0
            item_counts[item] += 1

    # Loop to keep only the items that meet the support threshold (Making L1)
    frequent_itemsets = {}
    for item, support in item_counts.items():
        if support >= min_support_count:
            frequent_itemsets[frozenset([item])] = support
    itemsets[1] = frequent_itemsets

    # Find the possible candidates(making C2) from the previous l(1)
    candidates = set()
    for combination in combinations(frequent_itemsets.keys(), 2):
        candidates.add(combination)

    # Initialize the count to 0 for the amount of pairs
    item_counts = {}
    for candidate in candidates:
        item_counts[candidate] = 0
         
    # Calculate the frequency of the pairs
    for transaction in transactions:
        for candidate in candidates:
            if set(candidate[0]).issubset(transaction) and set(candidate[1]).issubset(transaction):
                item_counts[candidate] += 1

    # Making L2
    frequent_itemsets = {}
    false_positive = defaultdict(int)
    for candidate, support in item_counts.items():
        if support >= min_support_count:
            frequent_itemsets[candidate] = support
        else:
            false_positive[candidate] += 1
                
    itemsets[2] = frequent_itemsets
    print("# of false positives: ", len(false_positive))


    return itemsets

In [8]:
# PCY Random sampling

def pcyrandom(df, min_support=0.01, perc=0.2):
    itemsets = {}
    # random sampling
    df_sampled = df.sample(frac=perc)
    transactions = list(map(set, df_sampled))
    num_transactions = len(transactions)
    min_support_count = min_support * num_transactions
    
    # PCY Pass 1
    # Making C1
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            if item not in item_counts:
                item_counts[item] = 0
            item_counts[item] += 1

    # Loop to keep only the items that meet the support threshold (Making L1)
    frequent_itemsets = {}
    for item, support in item_counts.items():
        if support >= min_support_count:
            frequent_itemsets[frozenset([item])] = support
    itemsets[1] = frequent_itemsets
            
    # Making C2
    candidates = set()
    for combination in combinations(frequent_itemsets.keys(), 2):
        candidates.add(combination)
    
    buckets = defaultdict(int)
    for transaction in transactions:
        for candidate in candidates:
            if set(candidate[0]).issubset(transaction) and set(candidate[1]).issubset(transaction):
                hash_value = (hash(candidate[0]) + hash(candidate[1])) % num_transactions
                buckets[hash_value] += 1
    
    # PCY Pass 2
    # Making L2
    frequent_pairs = {}
    false_positive = defaultdict(int)
    for pair, count in buckets.items():
        if count >= min_support_count:
            frequent_pairs[pair] = count    
        else:
            false_positive[pair] += 1
    itemsets[2] = frequent_pairs
    print("# of false positives: ", len(false_positive))

    return itemsets


In [9]:
# Apriori SON

def apriorison(df, min_support=0.01, perc=0.2):
    itemsets = {}
    transactions = list(map(set, df))
    num_transactions = len(transactions)
    min_support_count = min_support * num_transactions

    # Calculate the number of transactions to use based on percentage
    num_transactions_sampled = int(num_transactions * perc)

    # Divide transactions into chunks
    chunk_size = num_transactions_sampled // 2
    chunks = [transactions[i:i+chunk_size] for i in range(0, num_transactions_sampled, chunk_size)]

    # First pass to count item frequency in each chunk
    chunk_item_counts = [{} for _ in range(len(chunks))]
    for i, chunk in enumerate(chunks):
        for transaction in chunk:
            for item in transaction:
                if item not in chunk_item_counts[i]:
                    chunk_item_counts[i][item] = 0
                chunk_item_counts[i][item] += 1

    # Second pass to combine item counts from all chunks
    item_counts = {}
    for chunk_item_count in chunk_item_counts:
        for item, support in chunk_item_count.items():
            if item not in item_counts:
                item_counts[item] = 0
            item_counts[item] += support

    # Loop to keep only the items that meet the support threshold (Making L1)
    frequent_itemsets = {}
    for item, support in item_counts.items():
        if support >= min_support_count:
            frequent_itemsets[frozenset([item])] = support
    itemsets[1] = frequent_itemsets

    # Find the possible candidates(making C2) from the previous l(1)
    candidates = set()
    for combination in combinations(frequent_itemsets.keys(), 2):
        candidates.add(combination)

    # Initialize the count to 0 for the amount of pairs
    item_counts = {}
    for candidate in candidates:
        item_counts[candidate] = 0
         
    # Calculate the frequency of the pairs
    for chunk in chunks:
        for transaction in chunk:
            for candidate in candidates:
                if set(candidate[0]).issubset(transaction) and set(candidate[1]).issubset(transaction):
                    item_counts[candidate] += 1

    # Making L2
    frequent_itemsets = {}
    for candidate, support in item_counts.items():
        if support >= min_support_count:
            frequent_itemsets[candidate] = support
                
    itemsets[2] = frequent_itemsets

    return itemsets

In [10]:
# PCY SON

def pcyson(df, min_support=0.01, perc=0.2):
    itemsets = {}
    transactions = list(map(set, df))
    num_transactions = len(transactions)
    min_support_count = min_support * num_transactions
    perc_transactions = int(num_transactions * perc)
    
    # SON pass 1
    # Making C1
    item_counts = {}
    for transaction in transactions[:perc_transactions]:
        for item in transaction:
            if item not in item_counts:
                item_counts[item] = 0
            item_counts[item] += 1

    # Loop to keep only the items that meet the support threshold (Making L1)
    frequent_itemsets = {}
    for item, support in item_counts.items():
        if support >= min_support_count:
            frequent_itemsets[frozenset([item])] = support
    itemsets[1] = frequent_itemsets
            
    # Making C2
    candidates = set()
    for combination in combinations(frequent_itemsets.keys(), 2):
        candidates.add(combination)
    
    buckets = defaultdict(int)
    for transaction in transactions[:perc_transactions]:
        for candidate in candidates:
            if set(candidate[0]).issubset(transaction) and set(candidate[1]).issubset(transaction):
                hash_value = hash(candidate) % num_transactions
                buckets[hash_value] += 1
    
    # SON pass 2
    # Making L2
    frequent_pairs = {}
    for pair, count in buckets.items():
        if count >= min_support_count:
            frequent_pairs[pair] = count           
    itemsets[2] = frequent_pairs

    return itemsets


In [11]:
# multihash

def multihash(df, min_support=0.01, perc=0.2):
    itemsets = {}
    # random sampling
    df_sampled = df.sample(frac=perc)
    transactions = list(map(set, df_sampled))
    num_transactions = len(transactions)
    min_support_count = min_support * num_transactions
    
    # PCY Pass 1
    # Making C1
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            if item not in item_counts:
                item_counts[item] = 0
            item_counts[item] += 1

    # Loop to keep only the items that meet the support threshold (Making L1)
    frequent_itemsets = {}
    for item, support in item_counts.items():
        if support >= min_support_count:
            frequent_itemsets[frozenset([item])] = support
    itemsets[1] = frequent_itemsets
            
    # Making C2
    candidates = set()
    for combination in combinations(frequent_itemsets.keys(), 2):
        candidates.add(combination)
    
    # Pass 1 1st hash function
    buckets = defaultdict(int)
    for transaction in transactions:
        for candidate in candidates:
            if set(candidate[0]).issubset(transaction) and set(candidate[1]).issubset(transaction):
                hash_value = (hash(candidate[0]) + hash(candidate[1])) % num_transactions
                buckets[hash_value] += 1
                
    # Pass 2 2nd hash function
    buckets2 = defaultdict(int)
    for transaction in transactions:
        for candidate in candidates:
            if set(candidate[0]).issubset(transaction) and set(candidate[1]).issubset(transaction):
                hash_value = (hash(candidate[0]) * hash(candidate[1])) % num_transactions
                buckets2[hash_value] += 1
    
    # Pass 3 check if it is consistent in hash function 1 and 2
    # Making L2
    frequent_pairs = {}
    for pair, count in buckets.items():
        if count >= min_support_count and count in buckets2.values():
            frequent_pairs[pair] = count           
    itemsets[2] = frequent_pairs

    return itemsets

In [ ]:
# testing apriori random

start = time.time()
itemsets = apriorirandom(df, 0.05, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 20% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.05, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 40% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.05, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 60% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.05, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 80% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.05, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 100% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.02, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 20% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.02, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 40% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.02, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 60% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.02, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 80% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.02, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 100% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.01, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 20% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.01, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 40% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.01, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 60% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.01, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 80% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorirandom(df, 0.01, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 100% data and 1% support: ", elapsed, "seconds")

In [ ]:
# testing apriori son

start = time.time()
itemsets = apriorison(df, 0.05, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 20% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.05, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 40% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.05, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 60% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.05, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 80% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.05, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 100% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.02, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 20% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.02, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 40% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.02, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 60% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.02, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 80% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.02, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 100% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.01, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 20% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.01, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 40% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.01, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 60% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.01, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 80% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = apriorison(df, 0.01, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run Apriori Algorithm with 100% data and 1% support: ", elapsed, "seconds")

In [ ]:
# testing pcy random

start = time.time()
itemsets = pcyrandom(df, 0.05, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 20% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.05, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 40% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.05, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 60% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.05, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 80% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.05, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 100% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.02, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 20% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.02, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 40% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.02, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 60% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.02, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 80% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.02, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 100% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.01, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 20% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.01, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 40% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.01, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 60% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.01, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 80% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyrandom(df, 0.01, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 100% data and 1% support: ", elapsed, "seconds")

In [ ]:
# testing pcy son

start = time.time()
itemsets = pcyson(df, 0.05, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 20% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.05, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 40% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.05, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 60% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.05, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 80% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.05, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 100% data and 5% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.02, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 20% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.02, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 40% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.02, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 60% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.02, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 80% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.02, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 100% data and 2% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.01, 0.2)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 20% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.01, 0.4)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 40% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.01, 0.6)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 60% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.01, 0.8)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 80% data and 1% support: ", elapsed, "seconds")

start = time.time()
itemsets = pcyson(df, 0.01, 1)
end = time.time()
elapsed = end - start
print("Elapsed time to run PCY Algorithm with 100% data and 1% support: ", elapsed, "seconds")

In [13]:
# testing multihash
itemsets = multihash(df, 0.05, 1)
print("Frequent itemsets for 100% data and 5% support:")
for k, itemset_counts in itemsets.items():
    print("Itemsets of size {}: {}".format(k, len(itemset_counts)))
    
itemsets = multihash(df, 0.02, 1)
print("Frequent itemsets for 100% data and 2% support:")
for k, itemset_counts in itemsets.items():
    print("Itemsets of size {}: {}".format(k, len(itemset_counts)))
    
itemsets = multihash(df, 0.01, 1)
print("Frequent itemsets for 100% data and 1% support:")
for k, itemset_counts in itemsets.items():
    print("Itemsets of size {}: {}".format(k, len(itemset_counts)))
    
itemsets = multihash(df, 0.05, 0.2)
print("Frequent itemsets for 20% data and 5% support:")
for k, itemset_counts in itemsets.items():
    print("Itemsets of size {}: {}".format(k, len(itemset_counts)))
    
itemsets = multihash(df, 0.02, 0.2)
print("Frequent itemsets for 20% data and 2% support:")
for k, itemset_counts in itemsets.items():
    print("Itemsets of size {}: {}".format(k, len(itemset_counts)))
    
itemsets = multihash(df, 0.01, 0.2)
print("Frequent itemsets for 20% data and 1% support:")
for k, itemset_counts in itemsets.items():
    print("Itemsets of size {}: {}".format(k, len(itemset_counts)))

Frequent itemsets for 100% data and 5% support:
Itemsets of size 1: 6
Itemsets of size 2: 7
Frequent itemsets for 100% data and 2% support:
Itemsets of size 1: 20
Itemsets of size 2: 22
Frequent itemsets for 100% data and 1% support:
Itemsets of size 1: 70
Itemsets of size 2: 57
Frequent itemsets for 20% data and 5% support:
Itemsets of size 1: 6
Itemsets of size 2: 7
Frequent itemsets for 20% data and 2% support:
Itemsets of size 1: 19
Itemsets of size 2: 19
Frequent itemsets for 20% data and 1% support:
Itemsets of size 1: 72
Itemsets of size 2: 36
